In [23]:
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler

In [24]:
df = pd.read_csv('data_penjualan.csv')

df.head()

,Tanggal;Jenis Produk;Jumlah Order;Harga;Total
0,05/08/2022;Foodpak260;1000;1800;1800000
1,05/08/2022;FoodpakMatte245;1000;1900;1900000
2,05/08/2022;CraftLaminasi290;5000;750;3750000
3,05/08/2022;CraftLaminasi290;1000;1200;1200000
4,07/08/2022;Dupleks310;1000;1550;1550000


In [25]:
# Ubah kolom numerik
df[['Jumlah Order', 'Harga', 'Total']] = df[['Jumlah Order', 'Harga', 'Total']].apply(pd.to_numeric, errors='coerce')

KeyError: "None of [Index(['Jumlah Order', 'Harga', 'Total'], dtype='object')] are in the [columns]"

In [ ]:
# Agregasi per produk
data_produk = df.groupby('Jenis Produk')[['Jumlah Order', 'Harga', 'Total']].mean()
print(data_produk.head())

In [ ]:


# gunakan dataframe dari cell sebelumnya
data_produk = df

# -------- STANDARISASI TETAP PAKAI KOde MU SEBELUMNYA -----------
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_produk)

# ---------------- DIVISIVE CLUSTERING ---------------------------

def bisecting_kmeans(data, k):
    clusters = [(data, np.arange(len(data)))]  # cluster + index data

    while len(clusters) < k:
        # Pilih cluster dengan SSE (heterogenitas) terbesar
        sse_list = [np.sum((c[0] - c[0].mean(axis=0))**2) for c in clusters]
        idx = np.argmax(sse_list)

        data_to_split, index_to_split = clusters.pop(idx)

        # Pecah jadi dua cluster dengan KMeans
        km = KMeans(n_clusters=2, n_init='auto').fit(data_to_split)
        labels = km.labels_

        c1 = data_to_split[labels == 0]
        idx1 = index_to_split[labels == 0]

        c2 = data_to_split[labels == 1]
        idx2 = index_to_split[labels == 1]

        clusters.append((c1, idx1))
        clusters.append((c2, idx2))

    return clusters

# Tentukan jumlah cluster yang kamu mau
k = 3

# Jalankan divisive clustering
clusters = bisecting_kmeans(scaled_data, k)

# Ambil label untuk setiap produk
labels = np.zeros(len(scaled_data))

for cluster_id, (c, idx) in enumerate(clusters):
    labels[idx] = cluster_id

# Gabungkan kembali ke data_produk
data_produk['Cluster'] = labels.astype(int)

data_produk


ValueError: could not convert string to float: '05/08/2022;Foodpak260;1000;1800;1800000'

In [ ]:
plt.scatter(scaled_data[:,0], scaled_data[:,1], c=labels)
plt.title("Divisive Clustering (Bisecting K-Means)")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.show()
